In [1]:
# 라이브러리 정의 파트
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import xarray as xr
from mpl_toolkits import basemap
import os
import glob
import netCDF4

In [ ]:
import xarray as xr
import os

# ✅ 데이터 경로 설정
base_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA"
sgd_dir = os.path.join(base_dir, "SGD_TA")
obs_dir = os.path.join(base_dir, "OBS_TA")
mkprism_dir = os.path.join(base_dir, "MKPRISE_TA")

# ✅ 저장할 결과 파일 경로
output_dir = "/home/papalio/test_research/RMSE_TEST_2/PROG/create_data"
os.makedirs(output_dir, exist_ok=True)  # 디렉토리 없으면 생성
output_file = os.path.join(output_dir, "data_structure.txt")

# ✅ 확인할 샘플 날짜 (기본: 2020년 1월 1일)
sample_date = "20200101"

# ✅ 파일 경로 설정
sgd_file = os.path.join(sgd_dir, f"sfc_grid_ta_{sample_date}0000.nc")
obs_file = os.path.join(obs_dir, f"obs_ta_{sample_date}0000.nc")
mkprism_file = os.path.join(mkprism_dir, f"mkprism_ta_{sample_date}0000.nc")

# ✅ 데이터셋 리스트
datasets = {
    "SGD": sgd_file,
    "OBS": obs_file,
    "MK-PRISM": mkprism_file
}

# ✅ 데이터 구조 확인 및 저장 함수
def check_nc_structure(file_path, name):
    """NetCDF 파일의 구조 및 변수 정보 확인 후 저장"""
    output_lines = []

    if not os.path.exists(file_path):
        output_lines.append(f"🚨 {name} 데이터 파일이 존재하지 않습니다: {file_path}\n")
        return output_lines
    
    output_lines.append(f"✅ {name} 데이터 파일 확인됨: {file_path}\n")

    # ✅ NetCDF 데이터 열기
    ds = xr.open_dataset(file_path)

    # ✅ 데이터 구조 정보 저장
    output_lines.append(f"📌 {name} 데이터 구조\n")
    output_lines.append(str(ds))  # 전체 구조 저장
    output_lines.append("\n")

    # ✅ 변수 목록 출력
    output_lines.append("🔹 변수 목록:")
    for var in ds.data_vars:
        output_lines.append(f"   - {var}: {ds[var].dims}, dtype={ds[var].dtype}")
    output_lines.append("\n")

    # ✅ 속성 정보 저장
    output_lines.append("🔹 속성 정보:")
    for attr, value in ds.attrs.items():
        output_lines.append(f"   - {attr}: {value}")
    output_lines.append("\n")

    output_lines.append(f"🎉 {name} 데이터 구조 확인 완료!\n")
    output_lines.append("="*80 + "\n")  # 구분선

    return output_lines

# ✅ 결과 저장
with open(output_file, "w", encoding="utf-8") as f:
    for name, file_path in datasets.items():
        result_lines = check_nc_structure(file_path, name)
        f.write("\n".join(result_lines))

print(f"🎯 데이터 구조 확인 완료! 결과가 저장되었습니다: {output_file}")


In [ ]:
import xarray as xr
import os

base_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA"
datasets = {
    "OBS": os.path.join(base_dir, "OBS_TA"),
    "MKPRISE": os.path.join(base_dir, "MKPRISE_TA"),
}

for name, path in datasets.items():
    print(f"\n{name} dataset analysis:")
    files = sorted([f for f in os.listdir(path) if f.endswith(".nc")])
    
    for file in files[:5]:  # 샘플 5개만 체크
        file_path = os.path.join(path, file)
        ds = xr.open_dataset(file_path)
        print(f"{file}: dims={ds.dims}, dtype={ds['temperature'].dtype}, mean={ds['temperature'].values.mean()}")


In [2]:
import os
import numpy as np
import xarray as xr
import matplotlib.pyplot as plt
from multiprocessing import Pool

# ✅ 데이터 경로 설정
base_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA"
sgd_dir = os.path.join(base_dir, "SGD_TA")
obs_dir = os.path.join(base_dir, "OBS_TA")
mkprism_dir = os.path.join(base_dir, "MKPRISE_TA")

# ✅ 이미지 저장 경로 설정
img_base_dir = "/home/papalio/test_research/RMSE_TEST_2/IMG"
data_dir = os.path.join(img_base_dir, "test_data")

# ✅ 디렉토리 생성 (없으면 생성)
os.makedirs(data_dir, exist_ok=True)

# ✅ 데이터셋 리스트
datasets = {
    "SGD": sgd_dir,
    "OBS": obs_dir,
    "MKPRISE": mkprism_dir
}

# ✅ 개별 파일 처리 함수 (병렬 실행)
def process_file(args):
    """NetCDF 파일을 읽고 정상값, 결측값, 이상값, NaN 값을 카운트"""
    name, file_path = args
    try:
        ds = xr.open_dataset(file_path)

        # ✅ 변수 선택 (OBS & MK-PRISM은 `temperature` 사용)
        var_name = "temperature" if name in ["OBS", "MKPRISE"] else "data"
        if var_name not in ds:
            return None  # 변수 없음 -> 스킵

        # ✅ 데이터 로드 및 변환
        data_values = ds[var_name].values.astype(np.float32).flatten()

        # ✅ SGD 데이터 스케일 변환 (스케일 적용 필요)
        if name == "SGD":
            scale_factor = ds[var_name].attrs.get("data_scale", 1.0)
            data_values /= scale_factor

        # ✅ MK-PRISM 데이터 `float64 → float32` 변환
        if name == "MKPRISE":
            data_values = data_values.astype(np.float32)

        # ✅ NaN 값 카운트 (NaN을 제거하지 않고 계산에 포함)
        nan_count = np.sum(np.isnan(data_values))

        # ✅ 데이터 분류 (NaN 포함)
        missing_count = np.sum(data_values == -9990)  # 결측치 (-9990)
        normal_count = np.sum((data_values >= -30) & (data_values <= 70))  # 정상값 (-30~70)
        outlier_count = np.sum((data_values < -30) | (data_values > 70))  # 이상값

        return nan_count, missing_count, normal_count, outlier_count

    except Exception as e:
        print(f"⚠️ Error processing file {file_path}: {e}")
        return None

# ✅ 데이터셋별 병렬 처리
for name, data_path in datasets.items():
    print(f"📌 {name} data analysis started...")

    # ✅ 파일 목록 가져오기
    nc_files = sorted([f for f in os.listdir(data_path) if f.endswith(".nc")])
    file_paths = [(name, os.path.join(data_path, f)) for f in nc_files]

    if not file_paths:
        print(f"🚨 No {name} data found! ({data_path})")
        continue

    # ✅ 멀티프로세싱 실행
    with Pool(processes=4) as pool:  # CPU 코어 4개 사용
        results = pool.map(process_file, file_paths)

    # ✅ 유효한 결과만 필터링
    results = [r for r in results if r is not None]

    if not results:
        print(f"🚨 No valid {name} data for analysis!")
        continue

    # ✅ 결과 합산
    total_nan = sum(r[0] for r in results)
    total_missing = sum(r[1] for r in results)
    total_normal = sum(r[2] for r in results)
    total_outlier = sum(r[3] for r in results)

    # ✅ 파일이 이미 존재하면 스킵
    pie_chart_path = os.path.join(data_dir, f"{name}_data_distribution.png")
    if os.path.exists(pie_chart_path):
        print(f"⏩ {name} pie chart already exists. Skipping...")
        continue

    # ✅ Pie Chart 생성
    labels = ["Normal Data", "Missing Data (-9990)", "Outliers", "NaN Data"]
    sizes = [total_normal, total_missing, total_outlier, total_nan]
    colors = ["lightblue", "red", "purple", "gray"]

    plt.figure(figsize=(8, 8))
    plt.pie(
        sizes,
        labels=labels,
        autopct="%1.1f%%",
        colors=colors,
        startangle=140,
        wedgeprops={"edgecolor": "black"}
    )
    plt.title(f"{name} Data Distribution")
    
    # ✅ 이미지 저장
    plt.savefig(pie_chart_path, dpi=300)
    plt.close()

    print(f"📊 {name} data distribution pie chart saved: {pie_chart_path}")

print("\n🎉 Data analysis completed! Check the PNG files.")


📌 SGD data analysis started...
📊 SGD data distribution pie chart saved: /home/papalio/test_research/RMSE_TEST_2/IMG/test_data/SGD_data_distribution.png
📌 OBS data analysis started...
📊 OBS data distribution pie chart saved: /home/papalio/test_research/RMSE_TEST_2/IMG/test_data/OBS_data_distribution.png
📌 MKPRISE data analysis started...
📊 MKPRISE data distribution pie chart saved: /home/papalio/test_research/RMSE_TEST_2/IMG/test_data/MKPRISE_data_distribution.png

🎉 Data analysis completed! Check the PNG files.


In [1]:
import xarray as xr
import numpy as np
import os
from scipy.spatial import cKDTree
from datetime import datetime, timedelta
from multiprocessing import Pool, cpu_count

# ✅ 저장 경로 설정
obs_save_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA"
mkprism_save_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA/MKPRISE_TA"

os.makedirs(obs_save_dir, exist_ok=True)
os.makedirs(mkprism_save_dir, exist_ok=True)

# ✅ 원본 데이터 경로
base_sgd_dir = "/home/papalio/test_research/RMSE_TEST_2/DATA/SGD_TA"

# ✅ 변환할 날짜 목록 생성
start_date = datetime(2020, 1, 1)
end_date = datetime(2021, 12, 31)
dates = [start_date + timedelta(days=i) for i in range((end_date - start_date).days + 1)]

# ✅ 가상의 시도 중심 좌표 (17개 시도)
stations = {
    "서울": (37.5665, 126.9780),
    "부산": (35.1796, 129.0756),
    "대구": (35.8714, 128.6014),
    "인천": (37.4563, 126.7052),
    "광주": (35.1595, 126.8526),
    "대전": (36.3504, 127.3845),
    "울산": (35.5384, 129.3114),
    "세종": (36.4802, 127.2890),
    "경기": (37.4138, 127.5183),
    "강원": (37.8228, 128.1555),
    "충북": (36.6357, 127.4912),
    "충남": (36.6588, 126.6728),
    "전북": (35.7175, 127.1530),
    "전남": (34.8679, 126.9910),
    "경북": (36.5760, 128.5056),
    "경남": (35.4606, 128.2132),
    "제주": (33.4996, 126.5312)
}

# ✅ 파일 처리 함수 (OBS 변환)
def process_sgd_file(current_date):
    year, month, day = current_date.strftime("%Y"), current_date.strftime("%m"), current_date.strftime("%d")
    filename = f"sfc_grid_ta_{year}{month}{day}0000.nc"
    file_path = os.path.join(base_sgd_dir, filename)

    # ✅ 기존 변환 파일이 있으면 스킵
    obs_save_path = os.path.join(obs_save_dir, f"obs_ta_{year}{month}{day}0000.nc")
    if os.path.exists(obs_save_path):
        print(f"⚠️ {obs_save_path} 이미 존재함. 건너뜀.")
        return

    # ✅ 파일 존재 여부 확인
    if not os.path.exists(file_path):
        print(f"🚨 {file_path} 파일이 존재하지 않음. 건너뜀.")
        return

    print(f"✅ 변환 중: {file_path}")

    # ✅ NetCDF 파일 로드
    ds = xr.open_dataset(file_path)
    data = ds["data"].values.astype(np.float32)

    # ✅ 결측값 (-9990) 및 이상값 필터링
    data[data == -9990] = np.nan  # 결측값 처리
    data = data / ds["data"].attrs["data_scale"]  # 스케일 적용
    data[(data < -30) | (data > 70)] = np.nan  # 정상 범위 (-30°C ~ 70°C) 외 이상치 제거

    # ✅ 좌표 그리드 생성
    grid_size = ds.attrs["grid_size"]
    grid_nx, grid_ny = int(ds.attrs["grid_nx"]), int(ds.attrs["grid_ny"])
    map_slon, map_slat = ds.attrs["map_slon"], ds.attrs["map_slat"]

    lons = np.linspace(map_slon, map_slon + grid_size * (grid_nx - 1), grid_nx)
    lats = np.linspace(map_slat, map_slat + grid_size * (grid_ny - 1), grid_ny)
    lon_grid, lat_grid = np.meshgrid(lons, lats)

    # ✅ 최근접 관측소 데이터 추출 (OBS 변환)
    tree = cKDTree(list(zip(lat_grid.ravel(), lon_grid.ravel())))
    station_coords = np.array(list(stations.values()))
    _, idxs = tree.query(station_coords)

    obs_temps = data.ravel()[idxs]

    # ✅ NaN 값이 있으면 그대로 유지 (평균값 대체 X)
    if np.isnan(obs_temps).any():
        print(f"⚠️ {filename} OBS 변환 시 NaN 값 존재 → 그대로 유지")

    # ✅ OBS 데이터 저장
    obs_ds = xr.Dataset(
        {"temperature": (["station"], obs_temps)},
        coords={"station": list(stations.keys())}
    )
    obs_ds.to_netcdf(obs_save_path)
    print(f"📁 저장 완료: {obs_save_path}")

# ✅ 멀티프로세싱 실행
if __name__ == "__main__":
    num_workers = min(cpu_count(), 4)  # 최대 4개의 프로세스를 사용 
    print(f"🚀 멀티프로세싱 시작 (사용할 CPU 코어 수: {num_workers})")

    with Pool(num_workers) as pool:
        pool.map(process_sgd_file, dates)

    print("🎉 모든 변환 완료!")


🚀 멀티프로세싱 시작 (사용할 CPU 코어 수: 4)
⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202005180000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202004020000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202002160000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202001010000.nc 이미 존재함. 건너뜀.



⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202005190000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202002170000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202001020000.nc 이미 존재함. 건너뜀.


⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202004030000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202001030000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_202002180000.nc 이미 존재함. 건너뜀.⚠️ /home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA/obs_ta_2

In [2]:
#### obs data check ####

# setting path
path_obs = '/home/papalio/test_research/RMSE_TEST_2/DATA/OBS_TA'


# obs data sample check


# obs data value distribution